# Machine Learning with ToyotaHome/NVIDIA

## READ ME:
Run all cells in ascending order

### Setup/Imports

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
# Functions to use along widgets
from IPython.display import HTML, display, Markdown, Video, clear_output
from ipywidgets import Layout
# Faciliate file selection
from tkinter import *
from tkinter import filedialog
# Widget Packages
import ipywidgets as widgets
# jupyter nbextension enable --py widgetsnbextension
# Used for local directory
import os
import sys
# import TSU.json_util as json_util
# from TSU.json_util import test_train_json

In [ ]:
##Install dependencies

#!pip install -r requirements.txt

In [ ]:
## install pytorch
#pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

### Video Selector Functionality

In [ ]:
path = os.getcwd() # print(os.path.basename(path)) # get the local directory
videoSelector = widgets.Button(description = "Select Video Preview")
videoClearer = widgets.Button(description = "Clear Video Preview")

#Create the videoplayer here
video_output = widgets.Output() # Customisation

def video_Select(b):
    root = Tk()
    root.filename =  filedialog.askopenfilename(
        initialdir = path,title = "Select file",filetypes = (("videos","*.mp4"),("all files","*.*")))
    FileName = os.path.basename(root.filename)
    # Get directory path
    cwd = os.getcwd()
    # Get file path
    AbsoluteFilePath = os.path.normpath(root.filename)
    # Replace file path's directory nonsense with the relative command
    FilePath = AbsoluteFilePath.replace(cwd,".")
    root.destroy() #Closes the Window
    with video_output:
        # Create Video File
        video_output.clear_output()
        html_video_code = '<video width="100%" height="100%" controls><source src="{filepath}" type="video/mp4"></video>'.format(filepath = FilePath)
        video_output.append_display_data(HTML(html_video_code))

def video_Clear(b):
    with video_output:
        video_output.clear_output()

#Link the button to the function
videoSelector.on_click(video_Select)
videoClearer.on_click(video_Clear)

### Shared Widgets

In [ ]:
# selected_Directory = widgets.Text(
#     description = 'Directory',
#     value = '',
#     placeholder = 'Select Directory'
#     )
# current_Directory = widgets.Label()
# tooltip_Dir = widgets.Label("Current Directory: ")
# selectDir = widgets.Button(description = "Select Directory")
# clearDir = widgets.Button(description = "Clear")
# confirmDir = widgets.Button(description = "Confirm")

# sharedWidgets = widgets.VBox([widgets.HBox([tooltip_Dir,current_Directory]),widgets.HBox([selectDir, confirmDir, clearDir])])

# def directory_Select(b):
#     root = Tk()
#     root.directory =  filedialog.askdirectory (
#         initialdir = path)
#     selected_Directory.value = root.directory
#     root.destroy() #Closes the Window\n",

# def directory_Clear(b):
#     current_Directory.value = ""

# def directory_Print(b):
#     current_Directory.value = selected_Directory.value
# selectDir.on_click(directory_Select)
# clearDir.on_click(directory_Clear)
# confirmDir.on_click(directory_Print)

###  Toyota Widgets
These are the widgets required to run Toyota ML

In [ ]:
Toyota = widgets.IntSlider(description = "Toyo")

### NVIDIA Widgets
These are the widgets required to run NVIDIA ML

In [ ]:
Nvidia = widgets.IntSlider(description = "Nvidia")

In [ ]:
# Pre-requisites
"""
List of Machine Learning stuff
Consider making it a text file to store/edit?
"""
ml_list = ['Toyota Smart Home', 'Nvidia STEP']

# Widgets
mlSelectConfirm = widgets.Button(description="Confirm")
menu = widgets.Dropdown(
    options=ml_list,
    value=ml_list[0],
    description='ML Model:')

# Output
widgetset = widgets.Output()

# Function
def selectWidgetSet(b):
    with widgetset:
        widgetset.clear_output()
        print(menu.value + " is selected as the pipeline.")
#         if(menu.value == ml_list[0]): # if Toyota
#             display(Toyota, sharedWidgets)
#         elif(menu.value == ml_list[1]): # if Nvidia
#             display(Nvidia, sharedWidgets)


mlSelectConfirm.on_click(selectWidgetSet)

# Caption videos function

In [ ]:
import pandas
import cv2
import re
import os

class Captioning:
    def __init__(self, annotation_file_path, truth_file_path, video_file):
        self.sf = pandas.read_csv(annotation_file_path, header=0, names=['action','start','end','accuracy'], usecols=['action','start','end','accuracy'])[['action','start','end','accuracy']]
        self.df = pandas.read_csv(truth_file_path)
        self.video_path = video_file
        self.time = 0
        self.nextTime = 0
        self.nextActionTime = 0
        self.eventCounter = 0
        self.actionCounter = 0
        self.prediction = 0.0
        self.ground_truth = ""
        self.caption = ""
        self.output_path = "./videosCaptionOutput/"

    def getGroundTruth(self):
        if self.time == 0:
            self.nextTime = self.df['start_frame'].iloc[1]

        if self.time >= self.nextTime:
            try:
                self.ground_truth = self.df['event'].iloc[self.eventCounter]
                self.ground_truth =  re.sub('[_,.]', ' ', self.ground_truth)
                self.nextTime = self.df['start_frame'].iloc[self.eventCounter+1]
                self.eventCounter += 1
            except:
                pass
            
            
    def getCaption(self):
        if self.actionCounter == 0:
            self.actionCounter, = self.sf.index[self.sf['end'] == 'end']
            
        if self.time >= self.nextActionTime:
            self.actionCounter += 1
            try: 
                self.nextActionTime = float(self.sf['end'].iloc[self.actionCounter])
                self.caption = self.sf['action'].iloc[self.actionCounter]
                self.caption =  re.sub('[_,.]', ' ', self.caption)
                self.prediction = float(self.sf['accuracy'].iloc[self.actionCounter])
            except:
                pass


    def saveVideo(self):
        if not os.path.exists('./videosCaptionOutput/'):
            os.makedirs('./videosCaptionOutput/')
        cap = cv2.VideoCapture(self.video_path)
        # and our buffer to write frames
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_path+str(self.video_path[len(self.video_path)-13:len(self.video_path)-4])+'.mp4', fourcc, 25, (int(cap.get(3)),int(cap.get(4))))
        if (cap.isOpened() == False):
            print("Error opening video stream or file")

        while (cap.isOpened()):
            ret, frame = cap.read()

            if not ret:
                print(f'Video annotation is process complete.')
                break

            height, width, channels = frame.shape

            self.time = int(cap.get(cv2.CAP_PROP_POS_MSEC)/40)
            self.getGroundTruth()
            self.getCaption()
    
#             print(self.time, self.caption)
            cv2.rectangle(frame, (int(width * 0.05), int(height * 0.8)), (int(width * 0.95), int(height*0.95)), (159,159,159), -1)
            cv2.putText(frame, "Ground truth", (int(width*0.1),int(height*0.85)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, self.ground_truth, (int(width*0.3),int(height*0.85)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, "Prediction", (int(width*0.1),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, self.caption, (int(width*0.3),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, "Accuracy", (int(width*0.65),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, str(self.prediction), (int(width*0.8),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)

            #write our frame
            out.write(frame)
            cv2.imshow('frame',frame)

            key = cv2.waitKey(1)
            # define the key to
            # close the window
            if key == 'q' or key == 27:
                break

        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Usage Captioning(annotation_file_path, truth_file_path, video_file) 
# annotation_file_path (e.g. "./data/generatedAnnotations/PDAN_TSU_RGB_P02T01C06.csv" )
# truth_file_path (e.g. "./data/annotations/P02/P02T01C06.csv" )
# video_file (e.g. "./data/rgbVideos/P02T01C06.mp4")

# Pipeline selection

In [ ]:
title = widgets.Label("Pipeline selection")
tooltip = widgets.Label("Supported Video Types: MP4, WebM, and OGG.")

mlBox = widgets.VBox([widgets.HBox([menu, mlSelectConfirm]),widgetset])
toolBox = widgets.VBox([title,mlBox])

toolBox

### ML Widget Selector Functionality

# 1. Data Exploration

## 1.1 Select a video to playback

In [ ]:
# @hidden_cell
## retrieve all videos to dropdown list

def populateList(fileDirectory, fileType):
    folder_files = os.listdir(fileDirectory) #You can also use full path.
    print("This Folder contains {len_folder} file(s).".format(len_folder=len(folder_files)))
    fileList = []
    for file in folder_files:
        if file.endswith(fileType):
            fileList.append(file)
    return fileList

fileList = populateList('data/rgbVideos', ".mp4")
        
## dropdown list to select video to play
videoSelected = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    description="Video: ",
    disabled=False,
)
display(videoSelected)
#https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Button
videoButton = widgets.Button(
    description='Play video'
)
video_output = widgets.Output()
def explore_video(b):
    with video_output:
        # Create Video File
        video_output.clear_output()
        html_video_code = '<video width="80%" height="80%" controls><source src="./data/rgbVideos/{fileName}" type="video/mp4"></video>'.format(fileName = videoSelected.value)
        video_output.append_display_data(HTML(html_video_code))


videoButton.on_click(explore_video)

clearButton = widgets.Button(
    description='Clear Playback'
)

def clear_video(b):
    with video_output:
        video_output.clear_output()
        
# display(clearButton)
clearButton.on_click(clear_video)

## to align the buttons to 1 row
display(clearButton)
display(videoButton, video_output)

# 2. Inference Section

## Load a pretrained model

In [ ]:
model_dir = './TSU/models/'
# listdir() returns a list containing the names of the entries in the directory given by path.
modelList = os.listdir("./TSU/models")


style = {'description_width': 'initial'}

selectModel = widgets.Dropdown(
    options = modelList,
    value = modelList[0],
    description = "Choose a pre-trained model",
    style = style,
    disabled= False
)
# Output for dataset upload
selectModel
# print(selectModel.value)
select_button = widgets.Button(
    description='Select Trained Model'
)
display(selectModel,select_button)

# dropdown selected value --> selected_input_video.value
def get_selected_value(b):
    print(selectModel.value, "is selected as the pre-trained model for inferencing.")
select_button.on_click(get_selected_value)

## Choose an input video from the TSU project

In [ ]:
import os

## retrieve all TSU videos to dropdown list
folder_files = os.listdir('data/rgbVideos') 
print("This Folder contains {len_folder} file(s).".format(len_folder=len(folder_files)))
fileList=[]
for file in folder_files:
    fileList.append(file)

selected_input_video = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    description="Video: ",
    disabled=False,
)

select_button = widgets.Button(
    description='Select video'
)


display(selected_input_video, select_button)
# dropdown selected value --> selected_input_video.value
def get_selected_value(b):
    print(selected_input_video.value, "is selected as the video for inferencing.")
select_button.on_click(get_selected_value)

## Inference Results

In [ ]:
#see inference results in the form of output video with captions indicating detected activity in each video frame
from util_modules.json_util.test_train_json import inference_json

pretrain_model = selectModel.value
video_name = selected_input_video.value
dir = "./TSU/models/"+pretrain_model

#using train_test to just load 1 video
video_data = video_name.replace(".mp4","")
inference_output = inference_json(video_data, "TSU")
print("Inference dataset generated: ", inference_output)
'''
to run this command, essential parameters to add are:
 -video_name
 -load_model
Also do ensure you have a "generatedAnnotations" directory in the data folder 
in order to save the annotations.csv 
("./data/generatedAnnotations/")
'''

%run ./TSU/inferencing.py -load_model $dir -video_name $video_name -dataset "inference_smarthome_CS_51.json" -dataset_type "TSU"

In [ ]:
## display video with captions

## get sub folder name e.g. "P02" from video name
## dynamically retrieve annotations file based on user selected input
sub_folder = video_name[:3]
gt_annotation_directory = "./data/annotations/{folder}/".format(folder=sub_folder)
generated_annotation_directory = "./data/generatedAnnotations/"
annotation_file = video_name.replace(".mp4",".csv")
gen_annotation_file = "{loaded_model}_{vid_name}".format(loaded_model=pretrain_model, vid_name=annotation_file)
video_directory = "./data/rgbVideos/{vid_name}".format(vid_name=video_name)
print(gt_annotation_directory)
print(annotation_file)
print(gen_annotation_file)

test1 = Captioning(generated_annotation_directory+gen_annotation_file, gt_annotation_directory+annotation_file, video_directory)
test1.saveVideo()

# 3. Feature Extraction Section

In [ ]:
#! git clone https://github.com/v-iashin/video_features.git  ##need install manually, new version cant work

## Import necessary libraries

In [ ]:
%cd video_features/

In [ ]:
from video_features.utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch
from video_features.features_models.i3d.extract_i3d import ExtractI3D

 ## Initialise the extraction class

In [ ]:
class Extraction:
    
    def __init__(self, feature_type):
        self.args = OmegaConf.load(build_cfg_path(feature_type))
        
    def editExtraction(self, videos_txt, stream_type):
        # self.args.show_pred = "true"
        self.args.output_path = "../dataExtractionOutput"
        self.args.video_paths = videos_txt
        self.args.streams = stream_type
        self.args.flow_type = "raft"
        self.args.device = "cuda:0"
        self.args.file_with_video_paths = ""
        self.args.on_extraction = "save_numpy"
        
    def showArgs(self):
        print(self.args)
        
    def runExtraction(self):
        # Load the model
        extractor = ExtractI3D(self.args)

        # Extract features
        for video_path in self.args.video_paths:
            print(f'Extracting for {video_path}')
            # save to output
            extractor._extract(video_path)


## Choose video to extract npy Feature File

#### Remember to increase jupyter notebook config locally to access videos more than 10MB

In a new conda terminal type:
`jupyter notebook --generate-config`

Ctrl/Cmd F to replace tornado_settings to:
`c.NotebookApp.tornado_settings = {"websocket_max_message_size": 10000 * 1024 * 1024}`

Make sure to cd to the project root:
`cd ~`

Open your virtual environment:
`new_env\Scripts\Activate`

Run the jupyter notebook with the new configs:
`jupyter notebook --config="jupyter_notebook_config.py"`

##### Link is here https://github.com/jupyter-widgets/ipywidgets/issues/2522

In [ ]:
# @hidden_cell
## retrieve all videos to dropdown list

# Feature extraction title
feature_videos_label = widgets.HTML(
    value="<b>Upload your own videos to run feature extraction</b>",
)

# Allow user to upload own videos
feature_upload_videos = widgets.FileUpload(
    accept='.mp4',  # Currently only accepts .mp4
    multiple=True,
    description = "Browse",
    _counter = 0
)

# Choose the rgb/flow
stream_option_hint = widgets.HTML(
    value="(null = RGB+FLOW)",
)
stream_option_selection = widgets.Select(
    options=['null', 'rgb', 'flow'],
    value='null',
    description='Stream type:',
    disabled=False
)

# Confirm button
upload_feature_button = widgets.Button(
    description='Confirm Extraction',
    disabled = False
)

# retrieve function to run extraction tool
extracted = Extraction('i3d')

# Function to upload own dataset from computer
def upload_videos(file_val):
    if not file_val:
        print ("No file uploaded")

    else:   
        uploaded_video_filename =[]
        for item in file_val: 
            uploaded_video_filename.append("../data/rgbVideos/"+item["name"])
            print(str(uploaded_video_filename))
#             uploaded_video_filename = item["name"]

            # save the names
            extracted.editExtraction(uploaded_video_filename, stream_option_selection.value)
            extracted.runExtraction()
            print(f'{uploaded_video_filename} is extracted to "dataExtractionOutput" folder successfully!')
        
        try:
            
            # resets
            uploaded_file = ()
            uploaded_filename = ""
            content = ""
        except:
            print(sys.exc_info())
            

def on_feature_button_clicked(b):
    upload_videos(feature_upload_videos.value)
            

upload_feature_button.on_click(on_feature_button_clicked)

# User Interface
uploadFeatureBox = widgets.VBox([feature_videos_label,
                                 feature_upload_videos,
                                 widgets.HBox([stream_option_selection, stream_option_hint]),
                                 upload_feature_button
                                 ])

display(uploadFeatureBox)

# 4. Training Section

In [ ]:
%cd ../

## Selecting Dataset Code

In [ ]:
from util_modules.json_util.test_train_json import create_subset_json


# Training Dataset Upload Label
training_dataset_label = widgets.Label("Select dataset or upload your own")

# Create list for use with user dataset selection
datasetList = populateList("./TSU/tsu_data", ".json") # populate list with files from dataset directory

if not datasetList:
    print ("No existing datasets found")

# Allow user to select dataset
selectedDataset = widgets.Dropdown(
    options = datasetList,
    value= datasetList[0],
    description="Dataset: ",
    disabled=False,
)

# Allow user to upload own dataset
training_upload_dataset = widgets.Button(
    description='Upload dataset',
    disabled = False
)

# Allow user to cancel upload
training_upload_cancel = widgets.Button(
    description='Cancel',
    disabled = False
)
# Cancel upload functionality
def upload_dataset_cancel(b):
    with training_upload_output:
        clear_output()
        
# Link cancel button        
training_upload_cancel.on_click(upload_dataset_cancel)

# Output for dataset upload
training_upload_output = widgets.Output()

# Function to upload own dataset from computer
def upload_dataset(file_val):
    if not file_val:
        print ("No file uploaded")

    else:   
        clear_output(wait=True)
        uploaded_file = next(iter(file_val))
        uploaded_filename = uploaded_file["name"]
        content = uploaded_file["content"]
        print("File name: " + uploaded_filename)

        try:
            save_path = './TSU/tsu_data/'
            completeName = os.path.join(save_path, uploaded_filename) 
            with open(completeName, 'wb') as f: f.write(content)
            print (uploaded_filename + " uploaded to 'datasets' successfully!")
            
            # resets
            uploaded_file = ()
            uploaded_filename = ""
            content = ""
        except:
            print(sys.exc_info())
            display(training_upload_cancel)

# Upload functionality
def show_upload(b):
    with training_upload_output:
        clear_output()
        uploader = widgets.FileUpload(accept='.json',  # Currently only accepts .json
                                      multiple=False,
                                      description = "Browse",
                                      _counter = 0
        )
        upload_button = widgets.Button(
            description='Confirm Upload',
            disabled = False
        )

        def on_button_clicked(b):
            upload_dataset(uploader.value)

        upload_button.on_click(on_button_clicked)
        display(widgets.VBox([widgets.HBox([uploader, upload_button]),training_upload_cancel]))

#Link the buttons
training_upload_dataset.on_click(show_upload)


style = {'description_width': 'initial'}

# User Interface
uploadBox = widgets.VBox([training_dataset_label,
                          widgets.HBox([selectedDataset,training_upload_dataset]),
                         training_upload_output])

# Provide user with batch size
training_no = widgets.BoundedIntText(
    min=1,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='No. of training:',
    style=style,
    disabled=False
)

# Provide user with batch size
testing_no = widgets.BoundedIntText(
    min=1,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='No. of testing:',
    style=style,
    disabled=False
)

# Confirm Button to provide user with virtual commit
dataset_Confirm_Button = widgets.Button(
    description='Confirm',
    disabled = False
)
# Output to contain printout for ALL user input
datasetOutput = widgets.Output()


training_UserInput_label = widgets.Label("Current User Input")
# Functionality for user input
def on_button_click(b):
    with datasetOutput:
        json_output = create_subset_json(testing_no.value, training_no.value, selectedDataset.value, "./TSU/tsu_data/", "train_" +selectedDataset.value, "./TSU/tsu_data/")
        '''Print out the user input'''
        datasetOutput.clear_output() # Clear previous output
        print("Total train and test video data for training: ", json_output)

dataset_Confirm_Button.on_click(on_button_click)

dataset_input_box = widgets.HBox([widgets.VBox([uploadBox,training_no, testing_no,
                         dataset_Confirm_Button,datasetOutput])])
display(dataset_input_box)

## Set the Training Parameters

In [ ]:
# Things I need; UI = UserInput
'''
-load_model = ./models/PDAN_TSU_RGB [ui]
-mode = RGB [hardcoded]
'''

class user_input:
    def __init__(self):
        self.model_name = None
        self.batch_size = None
        self.epoch = None
        self.kernelsize = None
        self.mode = 'RGB'
        self.comp_info = 'TSU_CS_RGB_PDAN'
        self.train = True
        self.dataset = 'TSU'
        self.num_channel = 512
        self.APtype = 'map'
        self.model = 'PDAN'
        self.lr = 0.0002

        
modes = ['rgb','skeleton']
# Allow user to select type of TSU evaluation
training_video_mode = widgets.Dropdown(
    options = modes,
    value= modes[0],
    description="Video Type: ",
    disabled=False,
)
        

# Specify a name for this new model using appropriate UI elements.
desired_model_name = widgets.Text(
    placeholder='Type something..',
    description='Model Name:',
    disabled=False
)

# Create list for use with user dataset selection
datasetList = populateList("./TSU/models","") # populate list with files from dataset directory

if not datasetList:
    print ("No existing datasets found")


# Provide user with batch size
user_batch_size = widgets.BoundedIntText(
    min=1,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='Batch size:',
    disabled=False
)

# Provide user with epoch selection
epochs = widgets.BoundedIntText(
    min=1,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='Epoch:',
    disabled=False
)

# Provide user with kernel input
kernel = widgets.BoundedIntText(
    min=2,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='Kernel:',
    disabled=False
)

# Confirm Button to provide user with virtual commit
training_Confirm_Button = widgets.Button(
    description='Confirm',
    disabled = False
)

# Output to contain printout for ALL user input
trainingParamOutput = widgets.Output()

user = user_input() #Instantiate class here
training_UserInput_label = widgets.Label("Current User Input")
# Functionality for user input
def on_button_click(b):
    with trainingParamOutput:
        '''Store the user input into variables'''
        userModelName = desired_model_name.value
        userModelName = userModelName.replace(" ", "_") # strip spaces
        user.model_name = userModelName
        user.batch_size = user_batch_size.value
        user.epoch = epochs.value
        user.kernelsize = kernel.value
        user.mode = training_video_mode.value
        '''Print out the user input'''
        trainingParamOutput.clear_output() # Clear previous output
        for item in vars(user): # Clear previous output
            print("{}:{}".format(item,vars(user)[item]))

training_Confirm_Button.on_click(on_button_click)
inputBox = widgets.HBox([widgets.VBox([training_video_mode,
                         desired_model_name,
                         user_batch_size, epochs, kernel,
                         training_Confirm_Button]),widgets.VBox([training_UserInput_label,trainingParamOutput])])
display(inputBox)

## Run the training sequence, i.e., fit the model onto the dataset

In [ ]:
%cd ../

In [ ]:
# os.chdir('../')
print(os.getcwd())

In [ ]:
# Select Dataset
split_setting = ""
if "CS" in selectedDataset.value:
    split_setting = "CS"
elif "CV" in selectedDataset.value:
    split_setting = "CV"

bigRedButton = widgets.Button(
    description='Train Model',
    disabled = False,
    button_style='danger'
)

train_data = "train_" + selectedDataset.value
# print("Selected:",selectedDataset.value)

# Output train
trainOutput = widgets.Output()

# Functionality for user input
def runTrain(b):
    with trainOutput:
        clear_output()
        %run ./TSU/train.py -batch_size $user.batch_size \
        -model_name $user.model_name -epoch $user.epoch  \
        -kernelsize $user.kernelsize \
        -mode $user.mode -comp_info $user.comp_info -train $user.train \
        -dataset_type $user.dataset -num_channel $user.num_channel -APtype $user.APtype\
        -model $user.model -lr $user.lr -dataset $train_data

bigRedButton.on_click(runTrain)
display(widgets.VBox([bigRedButton,trainOutput]))

# 5. Testing Section

## Choose a dataset subfolder, using appropriate UI elements, from the data folder to use for testing

In [ ]:
# Choose a dataset subfolder, using appropriate UI elements, from the data folder to use for testing

# replicated from section 3
existingBox = widgets.VBox([label_1, selectedDataset])
newBox = widgets.VBox([label_2, selectown_button])
selectBox = widgets.HBox([existingBox, newBox])
display(selectBox)


## Load a pretrained model using an appropriate UI component

In [ ]:
# Load a pretrained model using an appropriate UI component

# replicated from section 2
selectModel
# print(selectModel.value)

## Run the testing sequence

In [ ]:
# Run the testing sequence, i.e., perform inference on each data sample and accumulate some statistics
# (again check out the train.py file)

## See some visual elements to indicate the progress of testing in the notebook

In [ ]:
# •	View some results 
# (e.g., Average Precision per activity class and mean Average Precision) 
# that allow for an assessment of how well the model performed

## Export results to csv 

In [ ]:
# Save the results to a results folder in the repo --> TBD

from TSU import test as testpy

def exportResultsOnClick(b):
    testpy.output_csv(indexArray, frames, args.video_name, args.load_model, val_map, val_loss, classAccuracy)
    print("working")
    
exportResultsButton = widgets.Button(
    description='Export results',
    disabled = False
)
exportResultsButton.on_click(exportResultsOnClick)
display(exportResultsButton)

# 7. New Pipeline (NVIDIA STEP)

In [ ]:
## STEP is just one other HOI project we suggest, but you are free to use any others. A good resource may be https://paperswithcode.com